In [9]:
%load_ext autoreload
%autoreload

from data.models import Coin
from django.db.models import Q
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
class Master:
    def __init__(self):
        pass
    
    @classmethod
    def inherited_from(cls, parent_cls):
        if parent_cls in cls.__mro__:
            return True
        else:
            raise cls.NotInheritedError()
            
    class NotInheritedError(Exception):
        pass

In [23]:
class Data(Master):
    def __init__(self):
        pass
    
    @property
    def _data_type(self):
        return None

    
class MarketData(Data):
    def get_close(self, *, start=None, end=None):
        return self._query(start=start, end=end, items=['close'])

    def get_high(self, *, start=None, end=None):
        return self._query(start=start, end=end, items=['high'])
    
    def get_low(self, *, start=None, end=None):
        return self._query(start=start, end=end, items=['low'])
    
    def get_open(self, *, start=None, end=None):
        return self._query(start=start, end=end, items=['open'])    

    def get_volume(self, *, start=None, end=None):
        return self._query(start=start, end=end, items=['volume'])    

    def get_amount(self, *, start=None, end=None):
        return self._query(start=start, end=end, items=['amount'])    
    
    def get_data(self, *, start=None, end=None, items=[]):
        return self._query(start=start, end=end, items=items)
        
    def _query(self, *, start=None, end=None, items=[]):
        raise NotImplementedError
    

class CoinMarketData(MarketData):
    def __init__(self, *, symbol, ccy, exchange):
        Coin.update(symbol=symbol, ccy=ccy, exchange=exchange)
        self.coin = Coin.objects.filter(symbol=symbol, ccy=ccy, exchange=exchange)
        super().__init__()
        
    def _query(self, *, start=None, end=None, items=[]):
        q = Q()
        if start is not None: 
            q.add(Q(time__gte=start), Q.AND)
            
        if end is not None:
            q.add(Q(time__lte=end), Q.AND)
        
        data = self.coin.filter(q)
        return pd.DataFrame(list(data.values('time', *items))).set_index('time').sort_index()

In [29]:
data = CoinMarketData(symbol='EOS', ccy='KRW', exchange='upbit')

2019-04-06 00:00:00 - 2019-06-28 07:00:00+00:00
2019-01-12 16:00:00 - 2019-04-05 23:59:59
2018-10-21 08:00:00 - 2019-01-12 15:59:59
2018-07-30 00:00:00 - 2018-10-21 07:59:59
2018-05-07 16:00:00 - 2018-07-29 23:59:59
2018-04-27 22:00:00 - 2018-05-07 15:59:59
completed


In [30]:
data.get_amount(start='2019-06-28 00:00:00+00:00')

,amount
time,
2019-06-28 00:00:00+00:00,1.838374e+09
2019-06-28 01:00:00+00:00,1.144387e+09
2019-06-28 02:00:00+00:00,1.458681e+09
2019-06-28 03:00:00+00:00,1.093478e+09
2019-06-28 04:00:00+00:00,1.136259e+09
2019-06-28 05:00:00+00:00,1.040851e+09
2019-06-28 06:00:00+00:00,1.601972e+09
2019-06-28 07:00:00+00:00,1.066724e+09
